In [1]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer
 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import f1_score,accuracy_score,classification_report
import sklearn.metrics as metrics
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import pickle as pk
from sklearn import svm

In [4]:
f = open('/home/carnd/BDCI/baseline/stopwords.txt','r', encoding='utf-8')
#stopwords_voacb = f.readline()
stopwords = []
for line in f.readlines():
    stopwords.append(line.strip())
#print(stopwords)

In [5]:
path = '/home/carnd/BDCI/baseline/'

In [6]:
def get_data():
    train = pd.read_csv(path + 'train.csv')
    test = pd.read_csv(path + 'test_public.csv')
 
    #将现有的数据随机打乱构成新的训练集，并在后面加上测试集
    train = train.sample(frac=1)
    train = train.reset_index(drop=True)
    data = pd.concat([train, test])
    
    #将lable用编码
    lbl =  LabelEncoder()
    lbl.fit(train['subject'])
    nb_classes = len(list(lbl.classes_))
    pk.dump(lbl, open('label_encoder.sav','wb'))
    
    #将数据的主题全部用数字表示
    subject = lbl.transform(train['subject'])
 
    #构建情感值label
    y = []
    for i in list(train['sentiment_value']):
        y.append(i)
 
    #构建主题label
    y1= []
    for i in subject:
        y1.append(i)
 
    print(np.array(y).reshape(-1,1)[:,0])
    return data,train.shape[0],np.array(y).reshape(-1,1)[:,0],test['content_id'],np.array(y1).reshape(-1,1)[:,0]

In [7]:
#对给定的字符串data用jieba分词
def processing_data(data):
    word = jieba.cut(data)
    return ' '.join(word)


In [8]:
def pre_process():
  
    data,nrw_train,y,test_id,y1 = get_data()
 
    #data['cut_commen']是分词之后的评论集
    data['cut_comment'] = data['content'].map(processing_data)
    
    #构建特征矩阵
    print('TfidfVectorizer')
    tf = TfidfVectorizer(ngram_range=(1,3),
                         token_pattern=r"(?u)\b\w+\b", 
                         analyzer='char',
                         stop_words=stopwords
                         )
    discuss_tf = tf.fit_transform(data['cut_comment'])
 
    print('HashingVectorizer')
    ha = HashingVectorizer(ngram_range=(1,2),
                           token_pattern=r"(?u)\b\w+\b",
                           lowercase=False,
                           stop_words=stopwords
                           )
    discuss_ha = ha.fit_transform(data['cut_comment'])
 
    data = hstack((discuss_tf,discuss_ha)).tocsr()
 
    return data[:nrw_train],data[nrw_train:],y,test_id,y1

In [9]:
X,test,y,test_id,y1= pre_process()


/home/carnd/anaconda3/envs/BDCI/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


[0 0 0 ... 0 0 0]


Loading model cost 0.905 seconds.
Prefix dict has been built succesfully.


TfidfVectorizer
HashingVectorizer


/home/carnd/anaconda3/envs/BDCI/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['一', '元', '吨', '数', '日', '末'] not in stop_words.
  sorted(inconsistent))


In [21]:
N = 10
kf = StratifiedKFold(n_splits=N, random_state=2018).split(X,y)

In [22]:
from sklearn.linear_model import LogisticRegression
#clf = LogisticRegression(C=0.5)
clf = svm.LinearSVC(loss='hinge', tol=1e-4, C=0.4)
clf_1 = svm.LinearSVC(loss='hinge', tol=1e-4, C=0.1)

In [23]:
y_train_oofp = np.zeros_like(y, dtype='float64')
y_train_oofp1 = np.zeros_like(y, dtype='float64')
 
y_test_oofp = np.zeros((test.shape[0], N))
y_test_oofp_1 = np.zeros((test.shape[0], N))

In [24]:
def micro_avg_f1(y_true, y_pred):
    return metrics.f1_score(y_true, y_pred, average='micro')


In [25]:
acc = 0
vcc = 0
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate,  label_1_train, label_1_validate,= X[train_fold, :], X[test_fold, :], y[train_fold], y[test_fold], y1[train_fold], y1[test_fold]
    clf.fit(X_train, label_train)
    
    val_ = clf.predict(X_validate)
    y_train_oofp[test_fold] = val_
    print('sentiment_value_f1:%f' % micro_avg_f1(label_validate, val_))
    acc += micro_avg_f1(label_validate, val_)
    result = clf.predict(test)
    y_test_oofp[:, i] = result
 

    clf_1.fit(X_train, label_1_train)
    val_1 = clf_1.predict(X_validate)
    y_train_oofp1[test_fold] = val_1
    
    print('topic_f1:%f' % micro_avg_f1(label_1_validate, val_1))
    vcc += micro_avg_f1(label_1_validate, val_1)
    result = clf_1.predict(test)
    y_test_oofp_1[:, i] = result
 
print(acc/N)
print(vcc/N)

sentiment_value_f1:0.719880
topic_f1:0.688755
sentiment_value_f1:0.722613
topic_f1:0.716583
sentiment_value_f1:0.730653
topic_f1:0.731658
sentiment_value_f1:0.715578
topic_f1:0.723618
sentiment_value_f1:0.722613
topic_f1:0.702513
sentiment_value_f1:0.713568
topic_f1:0.693467
sentiment_value_f1:0.731388
topic_f1:0.706237
sentiment_value_f1:0.735412
topic_f1:0.703219
sentiment_value_f1:0.726358
topic_f1:0.713280
sentiment_value_f1:0.726358
topic_f1:0.702213
0.7244421746316123
0.7081543914249355


In [26]:
lbl = pk.load(open('label_encoder.sav','rb'))
res_2 = []
for i in range(y_test_oofp_1.shape[0]):
    tmp = []
    for j in range(N):
        tmp.append(int(y_test_oofp_1[i][j]))
    word_counts = Counter(tmp)
    yes = word_counts.most_common(1)
    res_2.append(lbl.inverse_transform([yes[0][0]])[0])


In [27]:
res = []
for i in range(y_test_oofp.shape[0]):
    tmp = []
    for j in range(N):
        tmp.append(y_test_oofp[i][j])
    res.append(max(set(tmp), key=tmp.count))

In [28]:
print(len(res))
result = pd.DataFrame()
result['content_id'] = list(test_id)
 
result['subject'] = list(res_2)
result['subject'] = result['subject']
 
result['sentiment_value'] = list(res)
result['sentiment_value'] = result['sentiment_value'].astype(int)
 
result['sentiment_word'] = ''
result.to_csv('submit.csv',index=False)

2364
